In [6]:
#!pip install pandas nltk
#!pip install tqdm

In [201]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

In [202]:
input_file = "../data/toxic_comments_cleaned.csv"
output_file = "../data/processed.csv"
vocab_file = '../data/vocab.csv'
input_sample_file = "../data/toxic_comments_sample.csv";

Dataset should be downloaded from Here(https://www.kaggle.com/datasets/apoorvyadav/toxiccommentscleaned)

In [204]:
df = pd.read_csv(input_sample_file);

In [205]:
# for simplicity, easy handling
# df.to_csv("../data/toxic_comments_sample.csv");

In [254]:
# df = pd.read_csv(input_sample_file)

In [255]:
len(df)

2822

In [256]:
df.head()

,id,comment_text,target
0,0000997932d777bf,explanation why the edits made under my userna...,0.0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0


In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2822 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            2822 non-null   object 
 1   comment_text  2822 non-null   object 
 2   target        2822 non-null   float64
dtypes: float64(1), object(2)
memory usage: 152.7+ KB


In [258]:
(df[df["target"] == 1]["id"].count()) / df["id"].count()

np.float64(0.11020552799433027)

In [259]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bm2ic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bm2ic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bm2ic\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\bm2ic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [260]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [261]:
def clean_text(text):
    try:
        if not isinstance(text, str):
            return ''  # Return empty string if not a string

        # Remove special characters, numbers, and extra spaces
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        
        # Convert text to lowercase
        text = text.lower()
        
        # Tokenize text
        words = word_tokenize(text)
        
        # Remove stopwords
        words = [word for word in words if word not in stop_words]
        
        # Apply stemming
        stemmed_words = [stemmer.stem(word) for word in words]
        
        return ' '.join(stemmed_words)
        
    except Exception as e:
        print(f"Error processing text: {e}")
        return ''  # Return empty string in case of any error

In [262]:
df = df[(df['comment_text'].apply(lambda x: len(str(x).split())) <= 200) &
        (df['comment_text'].apply(lambda x: len(str(x)) <= 2000))]

In [263]:
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

In [264]:
df

,id,comment_text,target
0,0000997932d777bf,explanation why the edits made under my userna...,0.0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0
...,...,...,...
2995,08113e0533f6627a,new section at wpani there is now a new secti...,0.0
2996,081166fea250a5af,and asking top stop involving me,0.0
2997,08121cde0a727842,re all items i know that you said i did someth...,0.0
2998,08143ca4834f8bcf,so you not going tell me why you created so m...,0.0


In [265]:
# Load dataset in chunks
chunk_size = 10  # Process 1000 rows at a time

# Read CSV in chunks, we will take only a sample for simplicity
chunks = pd.read_csv(input_sample_file, chunksize=chunk_size)

# Enable tqdm progress bar
tqdm.pandas()

# Open output file to store processed data
with open(output_file, 'w', encoding='utf-8') as f_out:
    first_chunk = True  # To write headers only once

    for chunk in tqdm(chunks, desc="Processing chunks"):
        # Apply text cleaning function (NO recursion issues)
        # print(chunk['comment_text'] + "jdkd")
        chunk['cleaned_comment'] = chunk['comment_text'].apply(clean_text)
        # print(chunk['cleaned_comment'] + "fdfdf")

        # Append to CSV (write header only for the first chunk)
        chunk.to_csv(f_out, index=False, header=first_chunk, mode='a', encoding='utf-8')
        first_chunk = False  # Ensure only the first chunk writes headers

Processing chunks: 300it [00:04, 66.83it/s]


In [266]:
clean_text(df['comment_text'].iloc[150])

'unblock ill get lawyer block constitut right free speech'

In [267]:
processed_file = pd.read_csv("../data/processed.csv");


In [268]:
processed_file = processed_file.drop(columns=['Unnamed: 0'], errors='ignore')

In [269]:
processed_file

,id,comment_text,target,cleaned_comment
0,0000997932d777bf,explanation why the edits made under my userna...,0.0,explan edit made usernam hardcor metallica fan...
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0,daww match background colour im seemingli stuc...
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0,hey man im realli tri edit war guy constantli ...
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0,cant make real suggest improv wonder section s...
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0,sir hero chanc rememb page that
...,...,...,...,...
2995,08113e0533f6627a,new section at wpani there is now a new secti...,0.0,new section wpani new section wpani post compl...
2996,081166fea250a5af,and asking top stop involving me,0.0,ask top stop involv
2997,08121cde0a727842,re all items i know that you said i did someth...,0.0,item know said someth item lie honest tell rea...
2998,08143ca4834f8bcf,so you not going tell me why you created so m...,0.0,go tell creat mani account well say onlin priv...


In [ ]:
# Removing empty cleaned text comment
processed_file = processed_file.dropna(how='all')
processed_file = processed_file.dropna(subset=['cleaned_comment'])

In [271]:
processed_file.to_csv(output_file)


Tokenization cleaned comments

In [272]:
processed_file

,id,comment_text,target,cleaned_comment
0,0000997932d777bf,explanation why the edits made under my userna...,0.0,explan edit made usernam hardcor metallica fan...
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0,daww match background colour im seemingli stuc...
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0,hey man im realli tri edit war guy constantli ...
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0,cant make real suggest improv wonder section s...
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0,sir hero chanc rememb page that
...,...,...,...,...
2995,08113e0533f6627a,new section at wpani there is now a new secti...,0.0,new section wpani new section wpani post compl...
2996,081166fea250a5af,and asking top stop involving me,0.0,ask top stop involv
2997,08121cde0a727842,re all items i know that you said i did someth...,0.0,item know said someth item lie honest tell rea...
2998,08143ca4834f8bcf,so you not going tell me why you created so m...,0.0,go tell creat mani account well say onlin priv...


In [273]:
# Tokenize all comments
nltk.download('punkt', quiet=True);
processed_file['tokenized_comment'] = processed_file['cleaned_comment'].apply(word_tokenize);
print(processed_file['tokenized_comment'].head())


0    [explan, edit, made, usernam, hardcor, metalli...
1    [daww, match, background, colour, im, seemingl...
2    [hey, man, im, realli, tri, edit, war, guy, co...
3    [cant, make, real, suggest, improv, wonder, se...
4               [sir, hero, chanc, rememb, page, that]
Name: tokenized_comment, dtype: object


In [274]:
processed_file.to_csv(output_file)

In [284]:
import pandas as pd
import ast
from collections import Counter

# Define a function to process a chunk
def process_chunk(chunk):
    chunk['tokenized_comment'] = chunk['tokenized_comment'].apply(ast.literal_eval)
    return chunk

# Define a function to update the vocabulary
def update_vocab(vocab_counter, chunk):
    for sublist in chunk['tokenized_comment']:
        vocab_counter.update(sublist)

# Initialize counter for vocabulary
vocab_counter = Counter()

# Read data in chunks (adjust chunk size as needed)

chunk_size = 100  # Adjust based on available RAM

df_chunks = pd.read_csv(output_file, chunksize=chunk_size);  # Use chunksize to avoid loading entire file;

for chunk in df_chunks:
    chunk = process_chunk(chunk);  # Convert tokenized comments from string to list
    update_vocab(vocab_counter, chunk); # Update vocabulary counter

# Initialize vocabulary dictionary with PAD to ken set to ID 0
vocab = {'[PAD]': 0};

# Assign unique IDs to words
vocab.update({word: idx for idx, (word, _) in enumerate(vocab_counter.items(), start=1) if word != '[PAD]'});

print(f"Vocabulary size: {len(vocab)}");


Vocabulary size: 13402


In [276]:
# Saving vocab
vocab_df = pd.DataFrame(list(vocab.items()), columns=['word', 'id'])
vocab_df.to_csv(vocab_file, index=False)

In [277]:
# Load the vocabulary from CSV
vocab = pd.read_csv(vocab_file)
vocab = dict(zip(vocab['word'], vocab['id']))

In [278]:
# Function to convert tokens to word IDs
def words_to_ids(tokens, vocab):
    return [vocab.get(word) for word in tokens]
# Apply the word-to-ID conversion to the tokenized comments

# processed_file['sequence'] = processed_file['tokenized_comment'].apply(lambda tokens: words_to_ids(tokens, vocab))
# # Show the DataFrame with the word IDs
# print(processed_file[['tokenized_comment', 'sequence']].head())


In [285]:
import pandas as pd
import ast
import os

def process_in_place(file_path, vocab, chunk_size=10000):

    temp_file = file_path + ".tmp";

    with open(temp_file, 'w') as f:

        first_chunk = True;

        for chunk in pd.read_csv(file_path, chunksize=chunk_size):

            chunk['tokenized_comment'] = chunk['tokenized_comment'].apply(ast.literal_eval);

            chunk['sequence'] = chunk['tokenized_comment'].apply(lambda tokens: words_to_ids(tokens, vocab));

            chunk.to_csv(f, mode='a', header=first_chunk, index=False);

            first_chunk = False;

    # Replace the original file with the processed file
    os.replace(temp_file, file_path)
    print("Processing completed. File updated in-place.")


In [286]:
process_in_place(output_file, vocab);

Processing completed. File updated in-place.


In [287]:
processed_file = pd.read_csv(output_file)

In [288]:
processed_file

,id,comment_text,target,cleaned_comment,tokenized_comment,sequence
0,0000997932d777bf,explanation why the edits made under my userna...,0.0,explan edit made usernam hardcor metallica fan...,"['explan', 'edit', 'made', 'usernam', 'hardcor...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0,daww match background colour im seemingli stuc...,"['daww', 'match', 'background', 'colour', 'im'...","[27, 28, 29, 30, 25, 31, 32, 33, 22, 34, 35, 3..."
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0,hey man im realli tri edit war guy constantli ...,"['hey', 'man', 'im', 'realli', 'tri', 'edit', ...","[38, 39, 25, 40, 41, 2, 42, 43, 44, 20, 45, 46..."
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0,cant make real suggest improv wonder section s...,"['cant', 'make', 'real', 'suggest', 'improv', ...","[53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 6..."
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0,sir hero chanc rememb page that,"['sir', 'hero', 'chanc', 'rememb', 'page', 'th...","[91, 92, 93, 94, 23, 95]"
...,...,...,...,...,...,...
2994,08113e0533f6627a,new section at wpani there is now a new secti...,0.0,new section wpani new section wpani post compl...,"['new', 'section', 'wpani', 'new', 'section', ...","[14, 59, 6059, 14, 59, 6059, 360, 3891, 293, 3..."
2995,081166fea250a5af,and asking top stop involving me,0.0,ask top stop involv,"['ask', 'top', 'stop', 'involv']","[177, 165, 337, 2902]"
2996,08121cde0a727842,re all items i know that you said i did someth...,0.0,item know said someth item lie honest tell rea...,"['item', 'know', 'said', 'someth', 'item', 'li...","[2749, 79, 300, 358, 2749, 814, 1959, 523, 40,..."
2997,08143ca4834f8bcf,so you not going tell me why you created so m...,0.0,go tell creat mani account well say onlin priv...,"['go', 'tell', 'creat', 'mani', 'account', 'we...","[155, 523, 185, 380, 1388, 97, 600, 3528, 9780..."


In [289]:
processed_file.to_csv(output_file, index=False);


In [290]:
processed_file = pd.read_csv(output_file)
processed_file = processed_file.dropna(how='all')
processed_file.to_csv(output_file, index=False)


In [291]:
max_word_count = max(processed_file['comment_text'].apply(lambda x: len(str(x).split())))
print(max_word_count)


1087


In [292]:
max_index = processed_file['comment_text'].apply(lambda x: len(str(x).split())).idxmax()
print(processed_file.loc[max_index, 'comment_text'])  # Prints the comment with max words


fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot fuck you you useless bot 

In [293]:
processed_file

,id,comment_text,target,cleaned_comment,tokenized_comment,sequence
0,0000997932d777bf,explanation why the edits made under my userna...,0.0,explan edit made usernam hardcor metallica fan...,"['explan', 'edit', 'made', 'usernam', 'hardcor...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0,daww match background colour im seemingli stuc...,"['daww', 'match', 'background', 'colour', 'im'...","[27, 28, 29, 30, 25, 31, 32, 33, 22, 34, 35, 3..."
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0,hey man im realli tri edit war guy constantli ...,"['hey', 'man', 'im', 'realli', 'tri', 'edit', ...","[38, 39, 25, 40, 41, 2, 42, 43, 44, 20, 45, 46..."
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0,cant make real suggest improv wonder section s...,"['cant', 'make', 'real', 'suggest', 'improv', ...","[53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 6..."
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0,sir hero chanc rememb page that,"['sir', 'hero', 'chanc', 'rememb', 'page', 'th...","[91, 92, 93, 94, 23, 95]"
...,...,...,...,...,...,...
2994,08113e0533f6627a,new section at wpani there is now a new secti...,0.0,new section wpani new section wpani post compl...,"['new', 'section', 'wpani', 'new', 'section', ...","[14, 59, 6059, 14, 59, 6059, 360, 3891, 293, 3..."
2995,081166fea250a5af,and asking top stop involving me,0.0,ask top stop involv,"['ask', 'top', 'stop', 'involv']","[177, 165, 337, 2902]"
2996,08121cde0a727842,re all items i know that you said i did someth...,0.0,item know said someth item lie honest tell rea...,"['item', 'know', 'said', 'someth', 'item', 'li...","[2749, 79, 300, 358, 2749, 814, 1959, 523, 40,..."
2997,08143ca4834f8bcf,so you not going tell me why you created so m...,0.0,go tell creat mani account well say onlin priv...,"['go', 'tell', 'creat', 'mani', 'account', 'we...","[155, 523, 185, 380, 1388, 97, 600, 3528, 9780..."
